<a href="https://colab.research.google.com/github/frm1789/LanguageDetection/blob/main/Language_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install langdetect
!pip install langid
!pip install pycountry

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 22.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993221 sha256=e8bebd17764dd0093b2e3dc3579ebe3aa9b65275720fba7269c25d17933f7a7f
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 32.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langid: filename=langid-1.1.6-py3-none-any.whl size=1941171 sha256=5fb7b3df77ff7824efa5ba2d3882e531956dec1766d2be247c63adcf26789786
  Stored in directory: /root/.cache/pip/wheels/23/c8/c6/eed80894918490a175677414d40bd7c851413bbe03d4856c3c
Successfully built langid
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 13.4 MB/s eta 0:00:00


In [4]:
from langdetect import detect as langdetect_detect
from langdetect import LangDetectException
import langid
from transformers import pipeline
import pycountry

In [5]:
def get_language_name(language_code):
    try:
        language = pycountry.languages.get(alpha_2=language_code)
        return language.name if language else "Unknown"
    except KeyError:
        return "Unknown"

In [6]:
language_detector = pipeline("text-classification", model="papluca/xlm-roberta-base-language-detection")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [8]:
sentence = "It was the best of times, it was the worst of times, it was the age of wisdom, it was the age of foolishness, it was the epoch of belief, it was the epoch of incredulity, it was the season of light, it was the season of darkness, it was the spring of hope, it was the winter of despair."

In [9]:
# With langdetect
try:
    langdetect_code = langdetect_detect(sentence)
    langdetect_result = get_language_name(langdetect_code)
except LangDetectException:
    langdetect_result = "Detection failed"


In [10]:
# with langid
try:
    langid_code, langid_confidence = langid.classify(sentence)
    langid_result = {
        'language': get_language_name(langid_code),
        'confidence': langid_confidence
    }
except Exception as e:
    langid_result = "Detection failed"


In [11]:
# With Hugging Face's Transformers
try:
    hf_prediction = language_detector(sentence)[0]
    transformers_result = {
        'language': hf_prediction['label'],
        'confidence': hf_prediction['score']
    }
except Exception as e:
    transformers_result = "Detection failed"

In [12]:
# Print results
print("langdetect result:", langdetect_result)
print("langid result:", langid_result)
print("Transformers result:", transformers_result)

langdetect result: English
langid result: {'language': 'English', 'confidence': -1068.1853919029236}
Transformers result: {'language': 'en', 'confidence': 0.9366692900657654}
